# Unsupervised Learning

## Gensim LDA (Latent Dirichlet Allocation)

In [1]:
import numpy as np
import pandas as pd

import sys
import os

import gensim
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.matutils import corpus2csc

from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

import tqdm

from preprocess import preprocess, tokenizer

/home/andrewchen/PycharmProjects/COVID-Vaccine-Tweet/src/preprocess.py:111: DeprecationWarning: invalid escape sequence \.
  text = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", text).split())


In [2]:
data_root = '../data/cvt/'
df = pd.read_csv(os.path.join(data_root, 'covidvaccine.csv'))
df

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,hashtag_count
0,Ashwin,India 🇮🇳,I'm Just A Man On a Mission For My Vision!\n\n...,2018-01-08 06:01:00,89.0,3108.0,6780.0,0,2020-08-09 19:15:00,Huge Thanks And Best Wishes With @SerumInstInd...,covisheild,Twitter for Android,1
1,K. Kartik Rao,Mumbai and Nagpur India,Actor 🎭 \n\nhttps://t.co/u7GZfoWBDt\n\nSubscri...,2016-12-10 16:06:00,225.0,257.0,25362.0,0,2020-08-09 20:02:00,Any update of #CovidVaccine ????,covidvaccine,Twitter for Android,1
2,Mac Patrick (Road Hater) Henry,German Bundesrepublik,Liberty lover. \nServing the people of the 🇺🇸....,2014-01-11 19:47:00,338.0,277.0,28305.0,0,2020-08-09 21:21:00,From a private frontline COVID nurses group to...,covid19 covidvaccinerace covidvaccine pandemic,Twitter Web App,4
3,lydia Bell,Earth,"Mummy, animal lover , energy worker , awake in...",2009-03-23 01:09:00,2038.0,1105.0,63422.0,0,2020-08-09 21:29:00,So you want the whole world to take a #covidva...,covidvaccine,Twitter for iPhone,1
4,Lou Ann Donovan,"Kansas, USA",Live Young Look Young BE Young! Certified Inte...,2009-03-17 05:42:00,4365.0,4581.0,2505.0,0,2020-08-09 22:55:00,Watch to learn about the Phase 3 clinical tria...,covid19 covidvaccine covidclinicaltrial,Twitter for iPhone,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154533,Script Your Future '21,DMV,HowardU and Script Your Future has joined forc...,2021-01-31 18:40:41,15.0,42.0,4.0,0,2021-02-28 23:55:25,* Continue with routine preventive measures (s...,NaN,Twitter Web App,0
154534,eastvillagetwt,"East Village, New York, NY","Politics, the environment, science, culture, t...",2011-08-27 19:24:17,1968.0,5001.0,136110.0,0,2021-02-28 23:56:37,COVID: California Suspends System Of Shared Va...,covid19 covidvaccine,Twitter for iPhone,2
154535,Yogita,"New York, NY",Primum non nocere 💕👩🏽‍⚕️ | Tweets are my own. ...,2015-08-07 01:48:57,17.0,58.0,1194.0,0,2021-02-28 23:57:00,Today I got my first dose of the #COVIDVaccine...,covidvaccine,Twitter for Android,1
154536,DrZ,USA,NaN,2016-03-23 13:00:36,54.0,4.0,4.0,0,2021-02-28 23:57:06,Here's Where to Receive a COVID Vaccine in Ill...,illinois covidvaccine,Buffer,2


In [3]:
df['text'] = df['text'].apply(lambda x: preprocess(x))

df

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,hashtag_count
0,Ashwin,India 🇮🇳,I'm Just A Man On a Mission For My Vision!\n\n...,2018-01-08 06:01:00,89.0,3108.0,6780.0,0,2020-08-09 19:15:00,huge thanks best wishes seruminstindia adarpoo...,covisheild,Twitter for Android,1
1,K. Kartik Rao,Mumbai and Nagpur India,Actor 🎭 \n\nhttps://t.co/u7GZfoWBDt\n\nSubscri...,2016-12-10 16:06:00,225.0,257.0,25362.0,0,2020-08-09 20:02:00,update covidvaccine,covidvaccine,Twitter for Android,1
2,Mac Patrick (Road Hater) Henry,German Bundesrepublik,Liberty lover. \nServing the people of the 🇺🇸....,2014-01-11 19:47:00,338.0,277.0,28305.0,0,2020-08-09 21:21:00,private frontline covid nurses group today cov...,covid19 covidvaccinerace covidvaccine pandemic,Twitter Web App,4
3,lydia Bell,Earth,"Mummy, animal lover , energy worker , awake in...",2009-03-23 01:09:00,2038.0,1105.0,63422.0,0,2020-08-09 21:29:00,want whole world take covidvaccine s fast trac...,covidvaccine,Twitter for iPhone,1
4,Lou Ann Donovan,"Kansas, USA",Live Young Look Young BE Young! Certified Inte...,2009-03-17 05:42:00,4365.0,4581.0,2505.0,0,2020-08-09 22:55:00,watch learn phase clinical trial covid vaccine...,covid19 covidvaccine covidclinicaltrial,Twitter for iPhone,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154533,Script Your Future '21,DMV,HowardU and Script Your Future has joined forc...,2021-01-31 18:40:41,15.0,42.0,4.0,0,2021-02-28 23:55:25,continue routine preventive measures vaccinati...,NaN,Twitter Web App,0
154534,eastvillagetwt,"East Village, New York, NY","Politics, the environment, science, culture, t...",2011-08-27 19:24:17,1968.0,5001.0,136110.0,0,2021-02-28 23:56:37,covid california suspends system shared vaccin...,covid19 covidvaccine,Twitter for iPhone,2
154535,Yogita,"New York, NY",Primum non nocere 💕👩🏽‍⚕️ | Tweets are my own. ...,2015-08-07 01:48:57,17.0,58.0,1194.0,0,2021-02-28 23:57:00,today got first dose covidvaccine super thankful,covidvaccine,Twitter for Android,1
154536,DrZ,USA,NaN,2016-03-23 13:00:36,54.0,4.0,4.0,0,2021-02-28 23:57:06,heres receive covid vaccine illinois illinois ...,illinois covidvaccine,Buffer,2


In [4]:
df['text']

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0         huge thanks best wishes seruminstindia adarpoo...
1                                       update covidvaccine
2         private frontline covid nurses group today cov...
3         want whole world take covidvaccine s fast trac...
4         watch learn phase clinical trial covid vaccine...
                                ...                        
154533    continue routine preventive measures vaccinati...
154534    covid california suspends system shared vaccin...
154535     today got first dose covidvaccine super thankful
154536    heres receive covid vaccine illinois illinois ...
154537    utah slc sold walmart vaccine first dose book ...
Name: text, Length: 154538, dtype: object

In [5]:
# Prepare tokenized text
text_tokens = []
docs = [tokenizer(doc) for doc in list(df['text'])]
for doc in docs:
    text_tokens.append([token.orth_ for token in doc])

# Prepare corpus
id2word = Dictionary(text_tokens)
corpus = [id2word.doc2bow(doc) for doc in text_tokens]
word2id = id2word.token2id

# Prepare sparse corpus by converting gensim corpuss into sklearn sparse matrix
vectorized_content = corpus2csc(corpus).transpose().astype(int)

# Prepare feature names
feat_names = [id2word[k] for k in id2word.keys()]

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=text_tokens, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
already_run = False

# Topics range (length = 10)
min_topics = 2
max_topics = 22
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter (length = 5)
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter (length = 5)
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

if not already_run:
    # Can take a long time to run
    pbar = tqdm.tqdm(total=250)
    
    # Iterate through number of topics
    for k in topics_range:
        # Iterate through alpha values
        for a in alpha:
            # Iterate through beta values
            for b in beta:
                # Get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus, dictionary=id2word, k=k, a=a, b=b)
                # Save the model results
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)

                pbar.update(1)
    coherence_results = pd.DataFrame(model_results)
    coherence_results.to_csv(os.path.join(root, 'lda_tuning_results.csv'), index=False)
    pbar.close()
else:
    coherence_results = pd.read_csv(os.path.join(root, 'lda_tuning_results.csv'))

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
coherence_results

In [ ]:
# Sort coherence_results
sorted_coherence_results = coherence_results.sort_values(by=['Coherence'], ascending=False)
sorted_coherence_results

In [ ]:
# View the top coherence result
sorted_coherence_results.iloc[1]

In [ ]:
# Get alpha and beta from the best coherence result
hyper_params = sorted_coherence_results.iloc[1]
try: 
    alpha = float(hyper_params['Alpha']) 
except: 
    alpha = hyper_params['Alpha']

try:
    beta = float(hyper_params['Beta']) 
except:
    beta = hyper_params['Beta']

In [ ]:
lda = gensim.models.LdaMulticore(corpus=corpus, 
                                    id2word=id2word, 
                                    num_topics=int(hyper_params['Topics']), 
                                    random_state=100,
                                    chunksize=100,
                                    passes=10,
                                    alpha=alpha,
                                    eta=beta)

In [ ]:
# Print the 10 Keywords in the 4 topics
pprint(lda.print_topics())

In [ ]:
# Compute Coherence Score
lda_coherence = CoherenceModel(
    model=lda, texts=text_tokens, dictionary=id2word, coherence='c_v')
print('Coherence Score: ', lda_coherence.get_coherence())

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
lda_vis = pyLDAvis.gensim_models.prepare(lda, corpus, id2word)
lda_vis